<a href="https://colab.research.google.com/github/OhadSharet/Ass2/blob/master/masks_all_its.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Set-up


If running locally using jupyter, first install segment_anything in your environment using the installation instructions in the repository. If running from Google Colab, set using_colab=True below and run the cell. In Colab, be sure to select 'GPU' under 'Edit'->'Notebook Settings'->'Hardware accelerator'.

In [1]:
using_colab = True

In [2]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
    
    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
        
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-cvz15qd5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-cvz15qd5
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=f2c40b7853460a37b53c540e14bab4a5708c5f58c7283a22e5e5d6eea61b4159
  Stored in directory: /tmp/pip-ephem-wheel-cache-2wvauusr/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segm

##set-up

In [3]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import os


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    plt.imshow(img)

In [22]:
def get_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    return img

In [6]:
def get_all_files(directory):
	for entry in os.scandir(directory):
		if entry.is_file():
			yield entry.path
		elif entry.is_dir():
			yield from get_all_files(entry.path)

In [39]:
def get_last_directory_and_file(path):
    directory, filename = os.path.split(path)
    last_directory = os.path.basename(directory)
    result = os.path.join(os.sep, last_directory, filename)
    return result[1:]

In [28]:
def get_masks_path(img_path):
  masks_last_part = get_last_directory_and_file(img_path)
  return os.path.join(r"/content/drive/MyDrive/Colab Notebooks/files/masks",masks_last_part)

In [47]:
def load_image(image_path):
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, (0,0), fx = 0.2, fy = 0.2)
  return image


initialize SAM

In [8]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "/content/drive/MyDrive/Colab Notebooks/files/paths/sam_vit_b_01ec64.pth"
model_type = "vit_b"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

import an image

In [9]:
import os

In [57]:

for image_path in get_all_files(r"/content/drive/MyDrive/Colab Notebooks/files/it_images"):
  masks_path = get_masks_path(image_path)
  if os.path.exists(masks_path) or image_path.endswith("avi"):
    continue
  print(image_path)
  image = load_image(image_path)
  masks = mask_generator.generate(image)
  masked_img = get_anns(masks)
  plt.imsave(masks_path,masked_img)



save masks to file 


In [12]:
import pickle

In [14]:
with open('/content/drive/MyDrive/Colab Notebooks/files/pkls/masks1.pkl', 'wb') as fp:
    pickle.dump(masks, fp)
    print('dictionary saved successfully to file')


dictionary saved successfully to file


read data

In [17]:
with open('/content/drive/MyDrive/Colab Notebooks/files/pkls/masks1.pkl', 'rb') as fp:
    read_data = pickle.load(fp)


In [ ]:

  img = get_anns(read_data)

In [43]:
plt.imsave(r"/content/drive/MyDrive/Colab Notebooks/files/masks/test2.jpg",img)

In [32]:
p = "/content/drive/MyDrive/Colab Notebooks/files/it_images/PHOTO/CAM_2_4.jpg"

In [41]:
p1 = r"/content/drive/MyDrive/Colab Notebooks/files/masks"
full_p =get_masks_path(p)

In [42]:
print(full_p)

/content/drive/MyDrive/Colab Notebooks/files/masks/PHOTO/CAM_2_4.jpg


In [52]:
os.path.exists(r"/content/drive/MyDrive/Colab Notebooks/files/it_images/PHOTO/CAM_2_dfd4.jpg")

False